This notebook downsamples reanalysis dataset from 8,11 to 96,132

In [ ]:
!pip install function
!pip install netCDF4



In [ ]:
import xarray as xr
from netCDF4 import Dataset
# import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
# import cartopy.crs as ccrs  # for plotting map
# import cartopy
# import matplotlib as mpl
import numpy as np
import scipy.ndimage

# import tensorflow as tf


In [ ]:
# raeding reanalysis data

reanalysis = xr.open_dataset('finalprc2.nc')
# reanalysis = precipitation.isel(time=slice(1, None))
del reanalysis.attrs['history'] # remove long history text
lon = reanalysis.lon.values
lat = reanalysis.lat.values

time = np.arange(np.datetime64("1980-01-01T00"), np.datetime64("2011-01-01T00"), np.timedelta64(6, "h"))

In [ ]:
reanalysis

<xarray.Dataset>
Dimensions:  (time: 45292, lon: 3, lat: 2)
Coordinates:
  * time     (time) datetime64[ns] 1980-01-01 ... 2010-12-31T18:00:00
  * lon      (lon) float32 279.4 281.2 283.1
  * lat      (lat) float32 39.05 37.14
Data variables:
    prate    (time, lat, lon) float32 ...
Attributes:
    CDI:            Climate Data Interface version 1.9.10 (https://mpimet.mpg...
    Conventions:    COARDS
    title:          4x daily NMC reanalysis (1980)
    description:    Data is from NMC initialized reanalysis\n(4x/day).  It co...
    platform:       Model
    dataset_title:  NCEP-NCAR Reanalysis 1
    References:     https://www.psl.noaa.gov/data/gridded/data.ncep.reanalysi...
    CDO:            Climate Data Operators version 1.9.10 (https://mpimet.mpg...

In [ ]:
# IF REQUIRED TO TAKE SAMPLE

# wrf = wrf.isel(Time=slice(0,6000))
# reanalysis = reanalysis.isel(time=slice(0,6000))
# reforecast_apcp_c00 = reforecast_apcp_c00.isel(time=slice(0,6000))

In [ ]:
# bilinear interpolation for lat/lon
bilinear_lon = scipy.ndimage.zoom(lon, 12, order=1)
bilinear_lat = scipy.ndimage.zoom(lat, 12, order=1)
row_meshgrid, col_meshgrid = np.meshgrid(bilinear_lat, bilinear_lon, indexing='ij')

# bilinear interpolation for the reanalysis values
x = reanalysis.prate.values
x.shape
bilinear_gefs = scipy.ndimage.zoom(x, (1,12,12), order=1)

downscaled_reanalysis = xr.Dataset(
                data_vars=dict(
                    prate=(["time", "lon","lat"], bilinear_gefs, {"units":"kgm**-2"})
                ),
                coords=dict(
                    time=(["time"], time),
                    xlon=(["lon","lat"], col_meshgrid),
                    xlat=(["lon","lat"], row_meshgrid),
                ),
                attrs=dict(description="coords with matrices"),
            )

downscaled_reanalysis = downscaled_reanalysis.swap_dims({"lon":"lat", "lat":"lon"})

In [ ]:
y_train_hr = downscaled_reanalysis.sel(time=slice('1980-01-01T06', '2005-01-01T06'))
y_val_hr = downscaled_reanalysis.sel(time=slice('2006-01-01T12', '2008-12-31T12'))
y_test_hr = downscaled_reanalysis.sel(time=slice('2009-01-01T18', '2010-12-31T18'))

In [ ]:
y_train_hr

<xarray.Dataset>
Dimensions:  (time: 36529, lat: 24, lon: 36)
Coordinates:
  * time     (time) datetime64[ns] 1980-01-01T06:00:00 ... 2005-01-01T06:00:00
    xlon     (lat, lon) float32 279.4 279.5 279.6 279.7 ... 282.9 283.0 283.1
    xlat     (lat, lon) float32 39.05 39.05 39.05 39.05 ... 37.14 37.14 37.14
Dimensions without coordinates: lat, lon
Data variables:
    prate    (time, lat, lon) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
Attributes:
    description:  coords with matrices

In [ ]:
def timecheck(train, val, test):
    '''
    Return missing time step in train, val and test split
    '''
    train_timecheck = np.arange(np.datetime64("2000-01-01T06"), np.datetime64("2014-01-01T12"), np.timedelta64(6, "h"))
    val_timecheck = np.arange(np.datetime64("2014-01-01T12"), np.datetime64("2016-12-31T18"), np.timedelta64(6, "h"))
    test_timecheck = np.arange(np.datetime64("2016-12-31T18"), np.datetime64("2020-01-01T00"), np.timedelta64(6, "h"))
    print('train:', set(train_timecheck) - set(train.time.values.astype('datetime64[h]')))
    print('val:', set(val_timecheck) - set(val.time.values.astype('datetime64[h]')))
    print('test:', set(test_timecheck) - set(test.time.values.astype('datetime64[h]')))
    return

timecheck(y_train_hr, y_val_hr, y_test_hr) # no missing dates

train: {numpy.datetime64('2011-02-13T18','h'), numpy.datetime64('2007-05-20T12','h'), numpy.datetime64('2011-02-14T00','h'), numpy.datetime64('2007-05-20T18','h'), numpy.datetime64('2011-02-14T06','h'), numpy.datetime64('2007-05-21T00','h'), numpy.datetime64('2011-02-14T12','h'), numpy.datetime64('2007-05-21T06','h'), numpy.datetime64('2011-02-14T18','h'), numpy.datetime64('2007-05-21T12','h'), numpy.datetime64('2011-02-15T00','h'), numpy.datetime64('2007-05-21T18','h'), numpy.datetime64('2011-02-15T06','h'), numpy.datetime64('2007-05-22T00','h'), numpy.datetime64('2011-02-15T12','h'), numpy.datetime64('2007-05-22T06','h'), numpy.datetime64('2011-02-15T18','h'), numpy.datetime64('2007-05-22T12','h'), numpy.datetime64('2011-02-16T00','h'), numpy.datetime64('2007-05-22T18','h'), numpy.datetime64('2011-02-16T06','h'), numpy.datetime64('2007-05-23T00','h'), numpy.datetime64('2011-02-16T12','h'), numpy.datetime64('2007-05-23T06','h'), numpy.datetime64('2011-02-16T18','h'), numpy.datetime64(

In [ ]:
scaler_train_prate = MinMaxScaler()
y_one_col = y_train_hr.prate.values.reshape([y_train_hr.prate.values.shape[0]*y_train_hr.prate.values.shape[1]*y_train_hr.prate.values.shape[2], 1])
y_one_col = np.log10(y_one_col+1) # 10**X_one_col - 1 to scale back
y_one_col_res = scaler_train_prate.fit_transform(y_one_col) # scaler_train_apcp.inverse_transform(X_one_col_res) to scale back, or use 10**scaler_train_apcp.inverse_transform(X_one_col_res) -1 only
y_train_hr.prate.values = y_one_col_res.reshape(y_train_hr.prate.values.shape)

In [ ]:
def transform_val_test(val_test, scaler_train, is_prec=True):
    '''
    Input (example): ds_val_apcp.tp, scaler_train_apcp, True/False
    Output: Transformed validation/test XR data
    If is_prec set to True, variable is precipitation
    '''
    if is_prec == True:
        X_one_col = val_test.values.reshape([val_test.values.shape[0]*val_test.values.shape[1]*val_test.values.shape[2], 1])
        X_one_col = np.log10(X_one_col+1)
        X_one_col_res = scaler_train.transform(X_one_col)
        val_test.values = X_one_col_res.reshape(val_test.values.shape)
        return val_test.values

    else:
        X_one_col = val_test.values.reshape([val_test.values.shape[0]*val_test.values.shape[1]*val_test.values.shape[2], 1])
        # X_one_col = np.log10(X_one_col+1)
        X_one_col_res = scaler_train.transform(X_one_col)
        val_test.values = X_one_col_res.reshape(val_test.values.shape)
        return val_test.values

In [ ]:
# reanalysis
y_val_hr.prate.values = transform_val_test(y_val_hr.prate, scaler_train_prate, True)
y_test_hr.prate.values = transform_val_test(y_test_hr.prate, scaler_train_prate, True)

In [ ]:
y_val_hr

<xarray.Dataset>
Dimensions:  (time: 4381, lat: 24, lon: 36)
Coordinates:
  * time     (time) datetime64[ns] 2006-01-01T12:00:00 ... 2008-12-31T12:00:00
    xlon     (lat, lon) float32 279.4 279.5 279.6 279.7 ... 282.9 283.0 283.1
    xlat     (lat, lon) float32 39.05 39.05 39.05 39.05 ... 37.14 37.14 37.14
Dimensions without coordinates: lat, lon
Data variables:
    prate    (time, lat, lon) float32 0.0005411 0.0008417 ... 6.012e-05
Attributes:
    description:  coords with matrices

In [ ]:
# training FOR REANALYSIS DOWNSCALINGs
from google.colab import files
y_hr_train = y_train_hr.prate.values
y_hr_train = y_hr_train[..., np.newaxis]
print(y_hr_train.shape)
np.save('y_hr_train.npy', y_hr_train)

# val
y_hr_val = y_val_hr.prate.values
y_hr_val = y_hr_val[..., np.newaxis]
print(y_hr_val.shape)
np.save('y_hr_val.npy', y_hr_val)

# testing
y_hr_test = y_test_hr.prate.values
y_hr_test = y_hr_test[..., np.newaxis]
print(y_hr_test.shape)
np.save('y_hr_test.npy', y_hr_test)

files.download('y_hr_train.npy')
files.download('y_hr_val.npy')
files.download('y_hr_test.npy')


(36529, 24, 36, 1)
(4381, 24, 36, 1)
(2917, 24, 36, 1)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>